In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Chicago traffic crashes prediction

You've been given a crucial role as a data scientist for Chicago. 
Your job is to predict which accidents might require a response, like medical aid, towing, or both.
You'll analyze factors like accident location, road conditions, speed limits, and time. 
Chicago wants to use this information to better allocate its resources, considering factors like weather and time of day.

**Note**: This dataset is a small subset of the one available at the [Chicago Data Portal](https://data.cityofchicago.org/). 
We've chosen this subset because you'll be using a `KNeighborsClassifier`, which performs efficiently with small to medium-sized datasets but can be quite slow with larger ones. 
In future assignments, you'll work with the entire dataset.

In [27]:
# load data
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/traffic_crashes_Chicago.csv'
data = pd.read_csv(url, index_col='CRASH_DATE', parse_dates=True)
data

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,YEAR
CRASH_DATE,,,,,,,,,,,,,,,,,,,,,
2023-03-31 07:34:00,a0cdc2e317e24a87ffb5ed39a0f1ab99054fe04167615b...,JG205578,NaN,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,2.0,0.0,7,6,3,41.909494,-87.747824,POINT (-87.747823796021 41.909493550808),2023
2023-06-26 16:15:00,00e93310a117dc0228ee5e00affc77ab0bd3334e54db75...,JG317047,NaN,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,...,0.0,2.0,0.0,16,2,6,NaN,NaN,NaN,2023
2023-04-07 17:15:00,07c772b5d5b0264284f35a7769114ae681037a123d9872...,JG214567,NaN,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,PARKING LOT,...,0.0,2.0,0.0,17,6,4,41.834402,-87.616894,POINT (-87.61689418428 41.834401691989),2023
2023-02-01 16:00:00,4d0d885dfa2da00a8d196c58a8d4f249c3c697fb478ecb...,JG138027,NaN,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,2.0,0.0,16,4,2,41.962140,-87.645937,POINT (-87.645936592224 41.962140154293),2023
2023-02-27 09:55:00,2630202e4794a8b4dd665b5ad172b09f0be849937eb5f7...,JG167558,NaN,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,ONE-WAY,...,0.0,2.0,0.0,9,2,2,41.891604,-87.625307,POINT (-87.625306944978 41.89160410607),2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-23 08:41:00,39e3191301443098210c420c84157dec9a9fcd3b982f51...,JG271497,Y,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,2.0,0.0,8,3,5,41.707211,-87.628239,POINT (-87.628239101889 41.707211473793),2023
2023-04-21 21:15:00,ee4a15023569327d9ac20fe8a06dbc79aa4e353c7dcdf9...,JG232982,NaN,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,0.0,1.0,0.0,21,6,4,41.659773,-87.636651,POINT (-87.636650606697 41.659773314849),2023
2023-09-14 15:15:00,a0e267ee446b134cbdab5b9ae1f64c698a622c87984768...,JG424169,NaN,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,2.0,0.0,15,5,9,41.793506,-87.711398,POINT (-87.711398027946 41.793506266409),2023


In [28]:
# all object type df columns
list(data.select_dtypes(include=['object']).columns)

['CRASH_RECORD_ID',
 'RD_NO',
 'CRASH_DATE_EST_I',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'REPORT_TYPE',
 'CRASH_TYPE',
 'INTERSECTION_RELATED_I',
 'NOT_RIGHT_OF_WAY_I',
 'HIT_AND_RUN_I',
 'DAMAGE',
 'DATE_POLICE_NOTIFIED',
 'PRIM_CONTRIBUTORY_CAUSE',
 'SEC_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'PHOTOS_TAKEN_I',
 'STATEMENTS_TAKEN_I',
 'DOORING_I',
 'WORK_ZONE_I',
 'WORK_ZONE_TYPE',
 'WORKERS_PRESENT_I',
 'MOST_SEVERE_INJURY',
 'LOCATION']

Train, fine-tune, and test a `KNeighborsClassifier` model for predicting the `CRASH_TYPE` column in the dataset. Create a brief report summarizing your findings for the city of Chicago.

In [29]:
# drop columns with more than 90% missing values
data = data.dropna(thresh=0.9*len(data), axis=1)
data.isna().sum()

CRASH_RECORD_ID                    0
RD_NO                            719
POSTED_SPEED_LIMIT                 0
TRAFFIC_CONTROL_DEVICE             0
DEVICE_CONDITION                   0
WEATHER_CONDITION                  0
LIGHTING_CONDITION                 0
FIRST_CRASH_TYPE                   0
TRAFFICWAY_TYPE                    0
ALIGNMENT                          0
ROADWAY_SURFACE_COND               0
ROAD_DEFECT                        0
REPORT_TYPE                      506
CRASH_TYPE                         0
DAMAGE                             0
DATE_POLICE_NOTIFIED               0
PRIM_CONTRIBUTORY_CAUSE            0
SEC_CONTRIBUTORY_CAUSE             0
STREET_NO                          0
STREET_DIRECTION                   0
STREET_NAME                        0
BEAT_OF_OCCURRENCE                 0
NUM_UNITS                          0
MOST_SEVERE_INJURY                32
INJURIES_TOTAL                    32
INJURIES_FATAL                    32
INJURIES_INCAPACITATING           32
I

In [30]:
data.drop(['CRASH_RECORD_ID', 'RD_NO', 'LATITUDE', 'LONGITUDE', 'LOCATION'], axis=1, inplace=True)

/tmp/ipykernel_68136/4070424690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['CRASH_RECORD_ID', 'RD_NO', 'LATITUDE', 'LONGITUDE', 'LOCATION'], axis=1, inplace=True)


In [31]:
# your code here
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

knnclf = KNeighborsClassifier()

# classification pipeline
categorical_features = list(data.select_dtypes(include=['object']).columns)
# drop CRASH_TYPE from categorical_features
categorical_features.remove('CRASH_TYPE')

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
])

pipe_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn_clf', knnclf)
])

pipe_clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer())]),
                                                  ['TRAFFIC_CONTROL_DEVICE',
                                                   'DEVICE_CONDITION',
                                                   'WEATHER_CONDITION',
                                                   'LIGHTING_CONDITION',
                                                   'FIRST_CRASH_TYPE',
                                                   'TRAFFICWAY_TYPE',
                                                   'ALIGNMENT',
                                                   'ROADWAY_SURFACE_COND',
                                                   'ROAD_DEFECT', 'REPORT_TYPE',
                                                   'DAMAGE',
                                                   'DATE_POLICE_NOTIFIED',
                                                   'PRIM_CONTRIBUTORY_CAUSE',
                                                   'SEC_CONTRIBUTORY_CAUSE',
                                                   'STREET_DIRECTION',
                                                   'STREET_NAME',
                                                   'MOST_SEVERE_INJURY'])])),
                ('knn_clf', KNeighborsClassifier())])

In [34]:
data.columns

Index(['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'REPORT_TYPE', 'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'YEAR'],
      dtype='object')

In [33]:
data

,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,YEAR
CRASH_DATE,,,,,,,,,,,,,,,,,,,,,
2023-03-31 07:34:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,WET,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0.0,7,6,3,2023
2023-06-26 16:15:00,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0.0,16,2,6,2023
2023-04-07 17:15:00,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,PARKING LOT,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0.0,17,6,4,2023
2023-02-01 16:00:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0.0,16,4,2,2023
2023-02-27 09:55:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,WET,UNKNOWN,...,0.0,0.0,0.0,0.0,2.0,0.0,9,2,2,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-23 08:41:00,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0.0,8,3,5,2023
2023-04-21 21:15:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,1.0,0.0,21,6,4,2023
2023-09-14 15:15:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,1.0,0.0,2.0,0.0,15,5,9,2023


In [32]:
# grid params
param_grid = {
    'preprocessor__cat__encoder__handle_unknown': ['error', 'ignore'],
    'knn_clf__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'knn_clf__weights': ['uniform', 'distance'],
    'knn_clf__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# grid search
grid_clf = GridSearchCV(pipe_clf, param_grid, cv=5, scoring='accuracy')

# split data
X = data.drop(columns=['CRASH_TYPE'])
y = data['CRASH_TYPE']

X_train, X_test, y_train, y_test = train_test_split(X, y)

# fit grid search
grid_clf.fit(X_train, y_train)

# best params
grid_clf.best_params_

# best score
grid_clf.best_score_

# test score
grid_clf.score(X_test, y_test)

# confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = grid_clf.predict(X_test)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  F

KeyboardInterrupt: 